In [7]:
from pathlib import Path

def getAllOldCsv():
    output = []
    files = list(Path(
        "/Users/harshitagrawal/Desktop/projects/financial_freedom_projects/options_backtesting_bot/data/weekly_data/").rglob("*.[cC][sS][vV]"))
    for file in files:

        filepath = ("/".join(file.parts)
                    ).replace("//", "/")
        output.append(filepath)
    return output


oldFiles = getAllOldCsv()
print(oldFiles[0:5])

['/Users/harshitagrawal/Desktop/projects/financial_freedom_projects/options_backtesting_bot/data/weekly_data/2022-09-02_2022-09-08.csv', '/Users/harshitagrawal/Desktop/projects/financial_freedom_projects/options_backtesting_bot/data/weekly_data/2021-09-03_2021-09-09.csv', '/Users/harshitagrawal/Desktop/projects/financial_freedom_projects/options_backtesting_bot/data/weekly_data/2021-11-12_2021-11-18.csv', '/Users/harshitagrawal/Desktop/projects/financial_freedom_projects/options_backtesting_bot/data/weekly_data/2022-01-21_2022-01-27.csv', '/Users/harshitagrawal/Desktop/projects/financial_freedom_projects/options_backtesting_bot/data/weekly_data/2021-05-28_2021-06-03.csv']


In [25]:
import csv
from datetime import datetime, timedelta
from helper import generateFiles, writeFiles
from candleStick import CandleStickDataType, TimeFrameTypeEnum, CandleStickListDataType, InstrumentTypeEnum

def __readOptionsCsvFile(filename: str):
        output = []
        maxTimestamp = 0

        with open(filename, mode='r')as file:
            csvFile = csv.reader(file)
            for lines in csvFile:
                if lines[0].strip().lower() == '':
                    continue
                if lines[0].strip().lower() == 'ticker':
                    continue

                date = lines[1].strip().split(" ")[0]
                time = lines[1].strip().split(" ")[1]

                dateEntries = date.split("-")
                timeEntries = time.split(":")

                timestamp = datetime(
                    int(dateEntries[2]),
                    int(dateEntries[1]),
                    int(dateEntries[0]),
                    int(timeEntries[0]),
                    int(timeEntries[1]),
                    int(timeEntries[2]) if len(timeEntries) == 3 else 0
                ) + timedelta(seconds=19800)

                if int(timestamp.timestamp()) > maxTimestamp:
                    maxTimestamp = int(timestamp.timestamp())

                output.append(
                    {
                        "ticker": lines[0].lower().strip() if "BANKNIFTY" not in lines[0].lower().strip() else lines[0].lower().strip().replace("BANKNIFTY", "CNXBAN"),
                        "timestamp": int(timestamp.timestamp()),
                        "open": float(lines[2].lower().strip()),
                        "high": float(lines[3].lower().strip()),
                        "low": float(lines[4].lower().strip()),
                        "close": float(lines[5].lower().strip()),
                        "volume": int(lines[6].lower().strip()),
                        "open_interest": int(lines[7].lower().strip())
                    }
                )

        expiry = str(datetime.fromtimestamp(
            maxTimestamp).year) + '-' + str(datetime.fromtimestamp(
                maxTimestamp).month) + '-' + str(datetime.fromtimestamp(
                    maxTimestamp).day) + "T21:00:00.000"
        return output, expiry


def __convertOptionsCsvFile(fileData: dict, expiry: str) -> CandleStickListDataType:
    output = CandleStickListDataType(data=[])
    for entry in fileData:
        if "WK" not in entry["ticker"].upper():
            continue

        data = entry["ticker"].upper().split("WK")

        ticker = data[0]
        strike = int(data[1][0:-2])
        
        contract = ""
        if str(data[1]).replace(data[1][0:-2], "") == "CE":
            contract = "call"
        elif str(data[1]).replace(data[1][0:-2], "") == "PE":
            contract = "put"
        
        if contract == "":
            continue

        output.data.append(
            CandleStickDataType(
                ticker=ticker,
                open=entry["open"],
                high=entry["high"],
                low=entry["low"],
                close=entry["close"],
                timestamp=entry["timestamp"],
                iso_string=datetime.fromtimestamp(entry["timestamp"]).isoformat(),
                volume=entry["volume"],
                open_interest=entry["open_interest"],
                strike=strike,
                expiry_type="weekly",
                expiry=expiry,
                instrument=InstrumentTypeEnum.OPTIONS,
                contract_type=contract,
                timeframe=TimeFrameTypeEnum.MINUTE_1
            )
        )

    return output
        

fileData, expiryDate = __readOptionsCsvFile(
    '/Users/harshitagrawal/Desktop/projects/financial_freedom_projects/options_backtesting_bot/data/weekly_data/2022-09-02_2022-09-08.csv')

data = __convertOptionsCsvFile(fileData, expiryDate)

data = generateFiles(data, InstrumentTypeEnum.OPTIONS)

print(len(data))

writeFiles(data, InstrumentTypeEnum.OPTIONS)



1387
